In [1]:
import math
import numpy as np
import wandb

import torch
import torch_geometric
from torch_geometric.data import Data

import sys
import os
from tqdm import tqdm

# Add the 'scripts' directory to the Python path
scripts_path = os.path.abspath(os.path.join('..'))
if scripts_path not in sys.path:
    sys.path.append(scripts_path)
    
import joblib

# Now you can import the gnn_io module
import gnn_io as gio

import gnn_architectures as garch

## 1. Define model and parameters

In [2]:
# Define parameters 
num_epochs = 1000
unique_model_description = "mse_just_on_highways"
project_name = unique_model_description
path_to_save_dataloader = "../../data/data_created_during_training_needed_for_testing/"
indices_of_datasets_to_use = [0, 1, 2, 3]
batch_size = 16

data_dict_list = torch.load('../../data/train_data/dataset_1pm_0-2600_16_07.pt')

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
early_stopping = gio.EarlyStopping(patience=30, verbose=True)
torch.set_printoptions(precision=4, sci_mode=False)

## 2. Load data

In [3]:
# Reconstruct the Data objects
datalist = [Data(x=d['x'], edge_index=d['edge_index'], pos=d['pos'], y=d['y']) for d in data_dict_list]
dataset_only_relevant_dimensions = gio.cut_dimensions(dataset=datalist, indices_of_dimensions_to_keep=indices_of_datasets_to_use)
dataset_normalized = gio.normalize_dataset(dataset_only_relevant_dimensions)

In [4]:
baseline_error = gio.compute_baseline_of_no_policies(dataset_normalized)
print(f'Baseline error no policies: {baseline_error}')

baseline_error = gio.compute_baseline_of_mean_target(dataset_normalized)
print(f'Baseline error mean: {baseline_error}')

Baseline error no policies: 0.32162734866142273
Baseline error mean: 0.0032576548401266336


## 4. Train the model

We first find a good model for one batch. 

In [5]:
train_dl, valid_dl, test_dl = gio.create_dataloaders(batch_size = batch_size, dataset=dataset_normalized, train_ratio=0.7, val_ratio=0.15, test_ratio=0.15)
gio.save_dataloader(test_dl, path_to_save_dataloader + 'test_dl' + unique_model_description + '.pt')
gio.save_dataloader_params(test_dl, path_to_save_dataloader + 'test_loader_params' + unique_model_description+ '.json')

Total dataset length: 2580
Training subset length: 1805
Validation subset length: 387
Test subset length: 388


In [6]:
scaler_highways = joblib.load('x_scaler_dim_3.pkl')

In [7]:
highway_mapping = {
    'trunk': 0, 'trunk_link': 0, 'motorway_link': 0,
    'primary': 1, 'primary_link': 1,
    'secondary': 2, 'secondary_link': 2,
    'tertiary': 3, 'tertiary_link': 3,
    'residential': 4, 'living_street': 5,
    'pedestrian': 6, 'service': 7,
    'construction': 8, 'unclassified': 9,
    'np.nan': -1
}

higher_order_roads = ['primary', 'primary_link', 'secondary', 'secondary_link', 'tertiary', 'tertiary_link']

higher_order_values = {highway_mapping[road] for road in higher_order_roads if road in highway_mapping}

In [8]:
def get_indices_of_higher_order_roads(input_tensor, scaler, values_to_filter_for):
    values_dim3 = input_tensor[:, 3]
    original_highways_tensor = torch.tensor(scaler.inverse_transform(values_dim3.cpu().detach().numpy().reshape(-1, 1)).flatten(), dtype=torch.float)

    membership_mask = torch.tensor(np.isin(original_highways_tensor, list(values_to_filter_for)), dtype=torch.bool)
    indices = torch.nonzero(membership_mask).reshape(-1)
    return indices

In [9]:
def train(model, config=None, loss_fct=None, optimizer=None, train_dl=None, valid_dl=None, device=None, early_stopping=None):
    for epoch in range(config.epochs):
        model.train()
        actual_vals = []
        predictions = []
        for idx, data in tqdm(enumerate(train_dl)):
            input_node_features, targets = data.x.to(device), data.y.to(device)
            optimizer.zero_grad()
            
            # Extract values along the 3rd dimension
            indices = get_indices_of_higher_order_roads(input_tensor=input_node_features, scaler=scaler_highways, values_to_filter_for = higher_order_values)

            # Forward pass
            predicted = model(data)
            
            actual_vals.extend(targets.detach().numpy())
            predictions.extend(predicted.detach().numpy())
            
            filtered_predicted = torch.index_select(predicted, 0, indices)
            filtered_actual = torch.index_select(targets, 0, indices)
            
            # Backward pass
            train_loss = loss_fct(filtered_predicted, filtered_actual)
            
            train_loss.backward()
            optimizer.step()
            
            wandb.log({"train_loss": train_loss.item(), "epoch": epoch, "step": idx})
            # print(f"epoch: {epoch}, step: {idx}, loss: {train_loss.item()}")
        
        actual_vals = np.array(actual_vals)
        predictions = np.array(predictions)
        
        # Calculate R^2
        sst = ((actual_vals - actual_vals.mean()) ** 2).sum()
        ssr = ((actual_vals - predictions) ** 2).sum()
        r2 = 1 - ssr / sst

        val_loss = garch.validate_model_pos_features(model, valid_dl, loss_fct, device)
        print(f"epoch: {epoch}, validation loss: {val_loss}, R^2: {r2}")
        wandb.log({"loss": val_loss, "epoch": epoch, "r2": r2})
            
        early_stopping(val_loss)
        if early_stopping.early_stop:
            print("Early stopping triggered. Stopping training.")
            break
    
    print("Best validation loss: ", val_loss)
    wandb.summary["val_loss"] = val_loss
    wandb.finish()
    return val_loss, epoch

In [10]:
# Currently best architecture ! 
output_layer_parameter = 'gat'
hidden_size_parameter = 16
gat_layer_parameter = 0
gcn_layer_parameter = 0
lr = 0.001
in_channels = len(indices_of_datasets_to_use) + 2 # dimensions of the x vector + 2 (pos)

wandb.login()

wandb.init(
    project=project_name,
    config={
        "epochs": num_epochs,
        "batch_size": batch_size,
        "lr": lr,
        "early_stopping_patience": 10,
        "hidden_layer_size": hidden_size_parameter,
        "gat_layers": gat_layer_parameter,
        "gcn_layers": gcn_layer_parameter,
        "output_layer": output_layer_parameter,
        # "dropout": 0.15,
    }
)
config = wandb.config

print("output_layer: ", output_layer_parameter)
print("hidden_size: ", hidden_size_parameter)
print("gat_layers: ", gat_layer_parameter)
print("gcn_layers: ", gcn_layer_parameter)

gnn_instance = garch.MyGnn(in_channels=in_channels, out_channels=1, hidden_size=hidden_size_parameter, gat_layers=gat_layer_parameter, gcn_layers=gcn_layer_parameter, output_layer=output_layer_parameter)
model = gnn_instance.to(device)

best_val_loss, best_epoch = train(model, config=config, 
                                loss_fct=torch.nn.MSELoss(), 
                                optimizer=torch.optim.Adam(model.parameters(), lr=lr),
                                train_dl=train_dl, valid_dl=valid_dl,
                                device=device, early_stopping=early_stopping)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: enatterer (tum-traffic-engineering). Use `wandb login --relogin` to force relogin


output_layer:  gat
hidden_size:  16
gat_layers:  0
gcn_layers:  0
Model initialized
MyGnn(
  (pointLayer): PointNetConv(local_nn=Sequential(
    (0): Linear(in_features=6, out_features=16, bias=True)
    (1): ReLU()
    (2): Linear(in_features=16, out_features=16, bias=True)
  ), global_nn=Sequential(
    (0): Linear(in_features=16, out_features=8, bias=True)
    (1): ReLU()
    (2): Linear(in_features=8, out_features=32, bias=True)
    (3): ReLU()
    (4): Linear(in_features=32, out_features=16, bias=True)
  ))
  (output_layer): GATConv(16, 1, heads=1)
)


113it [02:23,  1.27s/it]


epoch: 0, validation loss: 0.003281985428184271, R^2: -8.60151481628418


113it [01:53,  1.00s/it]


...

epoch: 58, validation loss: 0.0033140592370182274, R^2: -0.013729453086853027
EarlyStopping counter: 14 out of 30


113it [02:08,  1.14s/it]


epoch: 59, validation loss: 0.003321602726355195, R^2: -0.018412470817565918
EarlyStopping counter: 15 out of 30


34it [00:43,  1.28s/it]


KeyboardInterrupt: 

In [ ]:
model_path = '../../data/trained_models/model_' + unique_model_description + '.pth'

# Save the model state dictionary and configuration
torch.save({
    'state_dict': model.state_dict(),
    'config': {
        'in_channels': model.in_channels,
        'out_channels': 1,
        'hidden_size': model.hidden_size,
        'gat_layers': model.gat_layers,
        'gcn_layers': model.gcn_layers,
        'output_layer': model.output_layer
    }
}, model_path)